I am using my home PC, with Debian 12 "bookworm" (testing) and an NVIDIA GPU.

The following setup is **not recommended** unless you know what you're doing with Linux, and probably not even then, but it's working okay for me.

I decided not to use conda, virtualenv, or docker. Instead, I installed the necessary Python modules under `/usr/local` with `pip`. This way I can use my AI tools directly from the command line and other scripts, like any other tools.

It's not safe to run `pip install -U` as root. It will merrily remove files from Debian-packaged Python modules under `/usr/lib`, and mess up the system. To avoid this, I changed permissions so that my regular user account can write to everything under `/usr/local` via the `staff` group:

```
sudo adduser $USER staff
sudo chgrp -R staff /usr/local
sudo chmod -R g+w /usr/local
```

This is somewhat of a security concern, but if a bad guy gets a local shell it's pretty much game over anyway. To make it safer, I could have used a different non-root account.

In order to upgrade a pip package where the same package was also installed in `/usr/lib` by dpkg, I need to use `pip -IU packagename`. The `-I` flag tells it to ignore installed packages, i.e. don't try to remove them. I only do that as needed for individual packages, when a normal upgrade fails.

This setup might upset Debian-packaged programs that depend on older versions of the Python libraries that I've upgraded. I haven't noticed any problems like that yet.

## This post is not complete yet!